In [1]:
from keras import backend as K
K.set_image_dim_ordering('tf')

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras import applications
from keras.utils.np_utils import to_categorical
from keras.models import load_model

import numpy as np
import pandas as pd
import cv2

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 224, 224

# top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'calorie_ranges/calorie_ranges_train/'
validation_data_dir = 'calorie_ranges/calorie_ranges_validation/'
nb_train_samples = 8438
nb_validation_samples = 1000
epochs = 60
batch_size = 10

In [3]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
model = applications.VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))

In [ ]:
def prepare_image(image_path):
    im = cv2.resize(cv2.imread(image_path), (224,224)).astype(np.float32)

    im[:,:,0] -= 103.939
    im[:,:,1] -= 116.779
    im[:,:,2] -= 123.68
    im = im.transpose((2,0,1))
    im = np.expand_dims(im, axis=0)
    return im

In [ ]:
synset = pd.read_csv('synset_words.txt', skipinitialspace=True, names = ['synset', 'words'])

In [ ]:
img.T[:50,:50,:,:].shape

In [ ]:
np.array([np.array([img.T[:,:,:,0]])]).shape

In [ ]:
img = prepare_image('burger.jpg')
out = new_model.predict(np.array([img.T[:,:,:,0]]))
y_pred = np.argmax(out)

print (out)

categorical_labels = to_categorical(out, num_classes=None)
print (categorical_labels)

print (y_pred)
# print (synset.loc[y_pred].synset)

In [ ]:
model.layers.pop()

In [ ]:
#freeze conv layers
for layer in model.layers:
    layer.trainable = False

In [ ]:
last_layer = model.output

x = Flatten()(last_layer)
# x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(75, activation='softmax')(x)

new_model = Model(model.input, predictions)
new_model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
                  loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
filepath="first1.food.weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [4]:
# train and valdiation generator and then model.fit_generator

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

Found 8434 images belonging to 5 classes.
Found 1008 images belonging to 5 classes.


In [ ]:
# fine-tune the model
new_model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks=callbacks_list)

# new_model.save_weights()

In [ ]:
# Running the new model with the saved best weights from the first training model to get better val accuracy 

In [ ]:
new_model.summary()

In [ ]:
new_model.save('first30_epochs_model.h5')

In [ ]:
model2 = load_model('first30_epochs_model.h5')

In [ ]:
filepath2 = "second2.food.weights.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list2 = [checkpoint2]

In [ ]:
model2.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks=callbacks_list2)

In [ ]:
model3 = load_model('75epochs_75categories_model.h5')

In [ ]:
filepath3 = "third.food.weights.hdf5"
checkpoint3 = ModelCheckpoint(filepath3, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list3 = [checkpoint3]

In [ ]:
model3.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks=callbacks_list3)

In [ ]:
model3.save('75epochs_75categories_model.h5')

In [7]:
classweights = {}

In [8]:
classweights[0] = float(1)
classweights[1] = float(3984/2509)
classweights[2] = float(3984/775)
classweights[3] = float(3984/457)
classweights[4] = float(3984/711)

In [ ]:
final_model = load_model('75epochs_75categories_model.h5')

In [ ]:
final_model.summary()

In [ ]:
last_layer = final_model.output

In [ ]:
last_layer = final_model.layers[-1].output
# final_model.get_layer(name='dense_2').name = 'dense_3'
better_predictions = Dense(5, activation='softmax')(last_layer)

In [ ]:
new_final_model = Model(final_model.input, better_predictions)

In [ ]:
new_final_model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
                  loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
filepath4 = "food.weights.hdf5"
checkpoint4 = ModelCheckpoint(filepath4, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list4 = [checkpoint4]

In [ ]:
new_final_model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks=callbacks_list4,
        class_weight = classweights)

In [ ]:
new_final_model.save('15epochs_calorie_classifying_model')

In [6]:
model = load_model('15epochs_calorie_classifying_model')

In [9]:
filepath4 = "75epochs_calorie_classifying_food.weights.hdf5"
checkpoint4 = ModelCheckpoint(filepath4, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list4 = [checkpoint4]

In [10]:
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks=callbacks_list4,
        class_weight = classweights)

Epoch 1/60


/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/Image.py:874: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


843/843 [==============================] - 387s - loss: 3.5732 - acc: 0.4075 - val_loss: 1.5123 - val_acc: 0.3240
Epoch 2/60
843/843 [==============================] - 387s - loss: 3.5508 - acc: 0.4231 - val_loss: 1.5073 - val_acc: 0.3367
Epoch 3/60
843/843 [==============================] - 381s - loss: 3.5456 - acc: 0.4156 - val_loss: 1.4943 - val_acc: 0.3417
Epoch 4/60
843/843 [==============================] - 381s - loss: 3.5349 - acc: 0.4211 - val_loss: 1.5123 - val_acc: 0.3206
Epoch 5/60
843/843 [==============================] - 382s - loss: 3.5258 - acc: 0.4281 - val_loss: 1.4905 - val_acc: 0.3437
Epoch 6/60
843/843 [==============================] - 381s - loss: 3.5131 - acc: 0.4292 - val_loss: 1.5023 - val_acc: 0.3226
Epoch 7/60
843/843 [==============================] - 381s - loss: 3.5050 - acc: 0.4254 - val_loss: 1.4748 - val_acc: 0.3687
Epoch 8/60
843/843 [==============================] - 381s - loss: 3.4887 - acc: 0.4337 - val_loss: 1.4765 - val_acc: 0.3477
Epoch 9/60


In [11]:
model.save('60epochs_calorie_classifying_model.h5')

In [ ]:
# Running restaurant images to get the 256 feature vectors to then do Supervised Learning

In [ ]:
model_practice = load_model('first30_epochs_model.h5')

In [ ]:
model_practice.layers.pop()
model_practice.layers.pop()

In [ ]:
list_of_features= []

In [ ]:
from os import listdir
from os.path import isfile, join

In [ ]:
list_image_names = [f for f in listdir('food_images/dennyspics/') if isfile(join('food_images/dennyspics/', f))]

In [ ]:
for image_name in list_image_names:
    
    file_path = 'food_images/dennyspics/'+image_name
    
    img = image.load_img(file_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0
    
    out = model_practice.predict(x)
    list_of_features.append(out[0].tolist())